### Redes Convolucionales y Keras

In [1]:
import keras
import tensorflow as tf

# no vamos a llamar los especificos para que tengamos claro de donde salen.
# esto nos provocara que tengamos que meternos en el esquema de las librerias keras 
# y tensorflow

2023-11-18 10:51:39.010731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 10:51:39.139346: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-18 10:51:39.642589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-18 10:51:39.642658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
from keras.datasets import mnist

In [3]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [4]:
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [5]:
#dimensiones de trabajo
img_rows,img_cols = 28,28

In [8]:
tf.keras.backend.image_data_format()

'channels_last'

In [7]:
batch_size = 128
num_classes=10
epochs=12

In [9]:
if tf.keras.backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols)
    x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols)
    input_shape = (1,img_rows,img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
    input_shape = (img_rows,img_cols,1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:',x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#convierte vectores de clase en binarios
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [11]:
## Primero, el modelo es secuencial
model = keras.models.Sequential()
units = 20
input_dim = input_shape
## agregamos capa convolucional 2D
model.add(
    tf.keras.layers.Conv2D(
        32,
        kernel_size=(3,3),
        activation='relu',
        input_shape = input_shape
    )
)

## agregamos segunda capa convolucional 2D
model.add(
    tf.keras.layers.Conv2D(
        64,
        kernel_size=(3,3),
        activation='relu'
    )
)

#model.add(
#    keras.layers.LSTM(units, input_shape=(None, input_dim))
#)
## agregamos capa de Pooling
model.add(
    tf.keras.layers.MaxPooling2D(
        pool_size=(2,2)
    )
)

## agregamos capa de regularizacion Dropout para minimizar calculos
model.add(
    tf.keras.layers.Dropout(
        0.25
    )
)

## agregamos capa de aplanado
model.add(
    tf.keras.layers.Flatten())

## agregamos capa de conexion completa
model.add(
    tf.keras.layers.Dense(
        units=128,
        activation='relu'
    )
)

## agregamos capa de regularizacion Dropout para minimizar calculos
model.add(
    tf.keras.layers.Dropout(
        0.5
    )
)

## agregamos capa de conexion completa
model.add(
    tf.keras.layers.Dense(
        units=num_classes,
        activation='softmax'
    )
)


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [ ]:
model.fit(
    x_train,y_train,
batch_size=batch_size,
epochs=epochs,
verbose = 1,
validation_data=(x_test,y_test)
)

score = model.evaluate(x_test,y_test,verbose = 0)
print('Test loss: ',score[0])
print('Test accuracy: ',score[1])

In [ ]:
#!pip install tensorflow-gpu
#import tensorflow as tf

In [ ]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


In [ ]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


In [ ]:
model.fit(
    x_train,y_train,
batch_size=batch_size,
epochs=epochs,
verbose = 1,
validation_data=(x_test,y_test))

score = model.evaluate(x_test,y_test,verbose = 0)
print('Test loss: ',score[0])
print('Test accuracy: ',score[1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_train[i], cmap=plt.get_cmap('gray'))
    plt.show()

In [ ]:
xpreds = model.predict(x_test)
xpreds.shape

In [ ]:
import numpy as np
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_test[i], cmap=plt.get_cmap('gray'))
    print(np.argmax(xpreds[i]), y_test[i])

In [ ]:
## Primero, el modelo es secuencial
model = keras.models.Sequential()

## agregamos capa convolucional 2D
model.add(
    tf.keras.layers.Conv2D(
        32,
        kernel_size=(3,3),
        activation='relu',
        input_shape = input_shape
    )
)

## agregamos segunda capa convolucional 2D
model.add(
    tf.keras.layers.Conv2D(
        64,
        kernel_size=(3,3),
        activation='relu'
    )
)

## agregamos capa de Pooling
model.add(
    tf.keras.layers.MaxPooling2D(
        pool_size=(2,2)
    )
)

## agregamos capa convolucional 2D
model.add(
    tf.keras.layers.Conv2D(
        64,
        kernel_size=(3,3),
        activation='relu',
        input_shape = input_shape
    )
)


## agregamos capa de Pooling
model.add(
    tf.keras.layers.MaxPooling2D(
        pool_size=(2,2)
    )
)

## agregamos capa de regularizacion Dropout para minimizar calculos
model.add(
    tf.keras.layers.Dropout(
        0.25
    )
)

## agregamos capa de aplanado
model.add(
    tf.keras.layers.Flatten())

## agregamos capa de conexion completa
model.add(
    tf.keras.layers.Dense(
        units=128,
        activation='relu'
    )
)

## agregamos capa de regularizacion Dropout para minimizar calculos
model.add(
    tensorflow.keras.layers.Dropout(
        0.5
    )
)

## agregamos capa de conexion completa
model.add(
    tensorflow.keras.layers.Dense(
        units=num_classes,
        activation='softmax'
    )
)


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [ ]:
model.fit(
    x_train,y_train,
batch_size=batch_size,
epochs=epochs,
verbose = 1,
validation_data=(x_test,y_test))

score = model.evaluate(x_test,y_test,verbose = 0)
print('Test loss: ',score[0])
print('Test accuracy: ',score[1])

In [ ]:
xpreds = model.predict(x_test)
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(x_test[i], cmap=plt.get_cmap('gray'))
    print(np.argmax(xpreds[i]), np.argmax(y_test[i]))

In [ ]:
y_test[100]

In [ ]:
arg = model.predict(x_test[[100]])
print(np.argmax(arg), y_test[i])

In [ ]:
plt.imshow(x_test[100], cmap=plt.get_cmap('gray'))

In [ ]:
file = '/media/tom/Fonte Storage/00 Clases/UDD/MLA/Laboratorios/IMG_0872.jpg'

In [ ]:
import cv2
#from PIL import Image

#im = Image.open(file)
im = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
plt.imshow(im, cmap=plt.get_cmap('gray'))

In [ ]:
im_resized = cv2.resize(im,(28,28),interpolation = cv2.INTER_LINEAR)
im_resized = cv2.bitwise_not(im_resized)

plt.imshow(im_resized,cmap='gray')

In [ ]:
im_resized = im_resized.astype('float32')
im_resized /= 255

imir = im_resized.reshape(1,img_rows,img_cols,1)
input_shape = (img_rows,img_cols,1)

imir = imir.astype('float32')
imir /= 255
print('x_train shape:',imir.shape)
print(imir.shape[0], 'train samples')

arg = model.predict(imir)
np.argmax(arg)